# L5: Dependency parsing

Dependency parsing is the task of mapping a sentence to a formal representation of its syntactic structure in the form of a dependency tree, which consists of directed arcs between individual words (tokens). In the lab you will implement a dependency parser based on the arc-standard algorithm and the fixed-window model that you implemented in Lab&nbsp;L4.

## The data set

The data set for this lab is the same as for Lab&nbsp;L4: the English Web Treebank from the [Universal Dependencies Project](http://universaldependencies.org). The code below defines an iterable-style dataset for parser data in the [CoNLL-U format](https://universaldependencies.org/format.html) that the project uses to distribute its data.

In [388]:
class Dataset():

    ROOT = ('<root>', '<root>', 0)  # Pseudo-root

    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename, 'rt', encoding='utf-8') as lines:
            tmp = [Dataset.ROOT]
            for line in lines:
                if not line.startswith('#'):  # Skip lines with comments
                    line = line.rstrip()
                    if line:
                        columns = line.split('\t')
                        if columns[0].isdigit():  # Skip range tokens
                            tmp.append((columns[1], columns[3], int(columns[6])))
                    else:
                        yield tmp
                        tmp = [Dataset.ROOT]

We load the training data and the development data:

In [389]:
train_data = Dataset('en_ewt-ud-train-projectivized.conllu')
dev_data = Dataset('en_ewt-ud-dev.conllu')

Both data sets consist of **parsed sentences**. A parsed sentence is represented as a list of triples, where the first component of each triple (a string) represents a word, and the second component (also a string) represents the word’s part-of-speech tag. The third component (an integer) specifies the position of the word’s syntactic head, i.e., its parent in the dependency tree. Run the following code cell to see an example:

In [390]:
example_sentence = list(train_data)[531]

example_sentence

[('<root>', '<root>', 0),
 ('I', 'PRON', 2),
 ('like', 'VERB', 0),
 ('yuor', 'PRON', 4),
 ('blog', 'NOUN', 2),
 ('.', 'PUNCT', 2)]

In this example the head of the pronoun *I* is the word at position&nbsp;2 – the verb *like*. The dependents of *like* are *I* (position&nbsp;1) and the noun *blog* (position&nbsp;4), as well as the final punctuation mark. Note that each sentence starts with the so-called **pseudo-root** (position&nbsp;0). This pseudo-root is a pseudo-word that is guaranteed to be the root of the dependency tree.

## Parser interface

Like the tagger in the previous lab, the parser that you will implement in this lab follows a simple interface:

In [391]:
class Parser(object):

    def predict(self, words, tags):
        raise NotImplementedError

The single method of this interface has the following specification:

**predict** (*self*, *words*, *tags*)

> Returns the list of predicted heads (a list of integers) for a single sentence, specified in terms of its *words* (a list of strings) and their corresponding *tags* (also a list of strings).

One trivial implementation of this interface is a parser that attaches each (real) word to its preceding word:

In [392]:
class TrivialParser(Parser):

    def predict(self, words, tags):
        return [0] + list(range(len(words)-1))

## Problem 1: Implement an evaluation function

Your first task is to implement a function that computes the **unlabelled attachment score (UAS)** of a parser on gold-standard data.

In [393]:
def uas(parser, gold_data):
    # TODO: Replace the next line with your own code
    correct = 0
    tot     = 0
    for sent in gold_data:
        words = []
        tags = []
        gold_standard_deps = []
        for word,  tag, dep in sent:
            words.append(word)
            tags.append(tag)
            gold_standard_deps.append(dep)
        pred = parser.predict(words, tags)
        
        
        for i in range(1, len(pred)):
            if gold_standard_deps[i] == pred[i]:
                correct += 1
            tot += 1    
    return correct / tot

Your implementation should conform to the following specification:

**uas** (*parser*, *gold_data*)

> Computes the unlabelled attachment score of the specified *parser* on the gold-standard data *gold_data* (an iterable of tagged sentences) and returns it as a float. The unlabelled attachment score is the percentage of all tokens to which the parser assigns the correct head (as per the gold standard). The calculation excludes the pseudo-roots.

In [394]:
uas(TrivialParser(), dev_data)

0.09757843254204938

### 🤞 Test your code

Test your code by computing the unlabelled attachment score for the trivial parser that attaches every word to its preceding word. The expected score on the development set is 9.76%.

## Problem 2: Create the vocabularies

The next cell contains skeleton code for a function `make_vocabs` that constructs the two vocabularies of the parser: one for the words and one for the tags. You should be quite familiar with this task by now. You will be able to re-use your code from lab&nbsp;L4.

In [395]:
PAD = '<pad>'
UNK = '<unk>'

def make_vocabs(gold_data):
    #print(list(gold_data))

    words = {}
    tags = {}

    curr_word_idx = 2
    curr_tag_idx = 1

    words[PAD] = 0
    words[UNK] = 1

    tags[PAD] = 0

    for sent in list(gold_data):
        for word, tag, dep in sent:
            if word not in words:
                words[word] = curr_word_idx
                curr_word_idx += 1
            
            if tag not in tags:
                tags[tag] = curr_tag_idx
                curr_tag_idx += 1

    return words, tags

Complete the code according to the following specification:

**make_vocabs** (*gold_data*)

> Returns a pair of dictionaries mapping the unique words and tags in the gold-standard data *gold_data* (an iterable over parsed sentences) to contiguous ranges of integers starting at zero. The word dictionary contains the pseudowords `PAD` (index&nbsp;0) and `UNK` (index&nbsp;1); the tag dictionary contains `PAD` (index&nbsp;0).

In [396]:
words, tags = make_vocabs(train_data)
print(len(words), len(tags))

19676 19


### 🤞 Test your code

Test your implementation by computing the total number of unique words and part-of-speech tags in the training data (including the pseudowords and the part-of-speech tag for the pseudoroot). The expected values are 19,676&nbsp;words and 19&nbsp;tags.

## Problem 3: Implement the arc-standard algorithm

The parser that you will implement in this lab consists of two parts: a static part that implements the logic of the arc-standard algorithm (presented in Lecture&nbsp;5.2), and a non-static part that contains the learning component – the fixed-window model that you implemented in Lab&nbsp;L4. In this problem you will implement the static part; the learning component is covered in Problem&nbsp;5.

Recall that, in the arc-standard algorithm, the next move (also called ‘transition’) of the parser is predicted based on features extracted from the current parser configuration, with references to the words and part-of-speech tags of the input sentence. On the Python side of things, the words and part-of-speech tags are represented as lists of strings, and a configuration is represented as a triple

$$
(i, \mathit{stack}, \mathit{heads})
$$

where $i$ is an integer specifying the position of the next word in the buffer, $\mathit{stack}$ is a list of integers specifying the positions of the words currently on the stack (with the topmost element last in the list), and $\mathit{heads}$ is a list of integers specifying the positions of the head words. If a word has not yet been assigned a head, its head value is&nbsp;0. To illustrate this representation, the initial configuration for the example sentence above is

and a possible final configuration is

**Note:** In Lecture&nbsp;5.2, both the buffer and the stack were presented as list of words. Here we only represent the *stack* as a list of words. To represent the *buffer*, we simply record the position of the next word that has not been processed yet (the integer $i$). This acknowledges the fact that the buffer (in contrast to the stack) can never grow, but will be processed from left to right.

The cell below contains a complete skeleton for the logic of the arc-standard algorithm:

In [397]:
from copy import deepcopy
class ArcStandardParser(Parser):

    MOVES = tuple(range(3))

    SH, LA, RA = MOVES  # Parser moves are specified as integers.

    @staticmethod
    def initial_config(num_words):
        # TODO: Replace the next line with your own code
        init = (0, [], [0] * num_words)
        return init

    @staticmethod
    def valid_moves(config):
        # TODO: Replace the next line with your own code
        valid_moves = []
        if config[0] != len(config[2]):
            valid_moves.append(0)
        
        if len(config[1]) >= 2:
            valid_moves.append(1)
            valid_moves.append(2)

        return valid_moves

    @staticmethod
    def next_config(config, move):
        # TODO: Replace the next line with your own code
        next_config = deepcopy(list(config))

        if move == 0:
            next_config[0] += 1
            next_config[1].append(next_config[0] - 1)
        if move == 1:
            dependant = next_config[1][len(next_config[1]) - 1 - 1]
            parent    = next_config[1][len(next_config[1]) - 1]  
            next_config[2][dependant] = parent                      # Assign parent to dependant in buffer
            next_config[1].pop(len(next_config[1]) - 1 - 1)         # Pop dependant from stack
        if move == 2:
            dependant = next_config[1][len(next_config[1]) - 1]
            parent    = next_config[1][len(next_config[1]) - 1 - 1]
            next_config[2][dependant] = parent                      # Assign parent to dependant in buffer
            next_config[1].pop(len(next_config[1]) - 1)             # Pop dependant from stack
        
        return tuple(next_config)

    @staticmethod
    def is_final_config(config):
        # TODO: Replace the next line with your own code
        if config[0] == len(config[2]) and len(config[1]) == 1:
            return True
        return False

Your implementation should conform to the following specification:

**initial_config** (*num_words*)

> Returns the initial configuration for a sentence with the specified number of words (*num_words*).

**valid_moves** (*config*)

> Returns the list of valid moves for the specified configuration (*config*).

**next_config** (*config*, *move*)

> Applies the *move* in the specified configuration *config* and returns the new configuration. This must not modify the input configuration.

**is_final_config** (*config*)

> Tests whether *config* is a final configuration.

### 🤞 Test your code

To test your implementation, you can run the code below. The code in this cell creates the initial configuration for the example sentence, simulates a sequence of moves, and then tests that the resulting configuration is the expected final configuration.

In [398]:
moves = [0, 0, 0, 1, 0, 0, 1, 2, 0, 2, 2]    # 0 = SH, 1 = LA, 2 = RA

parser = ArcStandardParser()
config = parser.initial_config(len(example_sentence))
for move in moves:
    assert move in parser.valid_moves(config)
    config = parser.next_config(config, move)
assert parser.is_final_config(config)
assert config == (6, [0], [0, 2, 0, 4, 2, 2])

print('Looks good!')

Looks good!


## Problem 4: Implement the oracle

The learning component of the parser is the next move classifier. To train this classifier, we need training examples of the form $(\mathbf{x}, m)$, where $\mathbf{x}$ is a feature vector extracted from a given parser configuration $c$, and $m$ is the corresponding gold-standard move. To obtain $m$, we need an **oracle**.

Recall that, in the context of transition-based dependency parsing, an oracle is a function that translates a gold-standard dependency tree (here represented as a list of head ids) into a sequence of moves such that, when the parser takes the moves starting from the initial configuration, then it recreates the original dependency tree. Here we ask you to implement the static oracle that was presented in Lecture&nbsp;5.2.

In [399]:
def oracle_moves(gold_heads):
    # TODO: Replace the next line with your own code

    parser = ArcStandardParser()
    config = parser.initial_config(len(gold_heads))

    occurances = {}
    for head in gold_heads:
        if head in occurances:
            occurances[head] += 1
        else:
            occurances[head] = 1
    
    while not parser.is_final_config(config):
        
        if len(config[1]) >= 2:
            stack = config[1]
            top        = stack[-1]
            second_top = stack[-2]

            # LA
            if top == gold_heads[second_top] and occurances.get(second_top, 0) == 0:
                yield config, 1
                config = parser.next_config(config, 1)
                occurances[top] -= 1
            # RA
            elif second_top == gold_heads[top] and occurances.get(top, 0) == 0:
                yield config, 2
                config = parser.next_config(config, 2)
                occurances[second_top] -= 1
            else:
                yield config, 0
                config = parser.next_config(config, 0)

        else:
            yield config, 0
            config = parser.next_config(config, 0)

Your implementation should conform to the following specification:

**oracle_moves** (*gold_heads*)

> Translates a gold-standard head assignment for a single sentence (*gold_heads*) into the corresponding stream of oracle moves. More specifically, this yields pairs $(c, m)$ where $m$ is a move (an integer, as specified in the `ArcStandardParser` interface) and $c$ is the parser configuration in which $m$ was taken.

### 🤞 Test your code

Test your code by running the cell below. This uses your implementation of *oracle_moves* to extract the oracle move sequence from the example sentence and compares it to the gold-standard move sequence *gold_moves*.

In [400]:
gold_heads = [h for w, t, h in example_sentence]
gold_moves = [0, 0, 0, 1, 0, 0, 1, 2, 0, 2, 2]

assert list(m for _, m in oracle_moves(gold_heads)) == gold_moves

## Problem 5: Fixed-window parser

Now it is time to put everything together. For the full implementation of the fixed-window parser, you will need the correspondents of the four parts of the fixed-window tagger from Lab&nbsp;L4: an implementation of the fixed-window model; a parser that uses the fixed-window model to make predictions; a function that generates the training examples for the parser; and the training loop.

### Problem 5.1: Implement the fixed-window model

The fixed-window model for the parser is the same as the fixed-window model for the tagger in Lab&nbsp;L4. You can simply copy your code from that lab.

In [401]:
import torch.nn as nn
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class FixedWindowModel(nn.Module):

    def __init__(self, embedding_specs, hidden_dim, output_dim):
        super().__init__()
        # TODO: Add your code here
        self.embedding_specs = embedding_specs
        self.embeddings = nn.ModuleList((nn.Embedding(i[1], i[2])) for i in embedding_specs for j in range(i[0]))  

        self.linear1   = nn.Linear(50*3 + 10 * 3, hidden_dim)
        self.relu      = nn.ReLU()
        self.linear2   = nn.Linear(hidden_dim, output_dim)
        
    # Features is B x k where B == batch_size and k is total embeddings (k = 4, 3 word embs and 1 tag emb)
    def forward(self, features):
        # TODO: Replace the next line with your own code
        #print(self.embeddings)
        #print("feature size: ", features.size())

        feat_v = [self.embeddings[i](features[..., i]) for i in range(0, len(self.embeddings))]         
        feat_v = torch.cat(feat_v, -1)  
        
        linear1 = self.linear1(feat_v)
        rel     = self.relu(linear1)
        output  = self.linear2(rel)   

        return output

In [402]:
embedding_specs = [(3, len(words), 50), (3, len(tags), 10)]
FixedWindowModel(embedding_specs, 180, 3)

FixedWindowModel(
  (embeddings): ModuleList(
    (0): Embedding(19676, 50)
    (1): Embedding(19676, 50)
    (2): Embedding(19676, 50)
    (3): Embedding(19, 10)
    (4): Embedding(19, 10)
    (5): Embedding(19, 10)
  )
  (linear1): Linear(in_features=180, out_features=180, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=180, out_features=3, bias=True)
)

### Problem 5.2: Implement the parser

The next step is to implement the parser itself. This parser will use the fixed-window model to predict the next move for a given configuration in the arc-standard algorithm, based on the features extracted from the current feature window.

#### Default feature model

For the parser, we ask you to implement a fixed-window model with the following features ($k=6$):

0. word form of the next word in the buffer
1. word form of the topmost word on the stack
2. word form of the second-topmost word on the stack
3. part-of-speech tag of the next word in the buffer
4. part-of-speech tag of the topmost word on the stack
5. part-of-speech tag of the second-topmost word on the stack

Whenever the value of a feature is undefined, you should use the special value `PAD`.

#### Hyperparameters

The following choices are reasonable defaults for the hyperparameters of the network architecture used by the parser:

* width of the word embedding: 50
* width of the tag embedding: 10
* size of the hidden layer: 180

In [403]:
class FixedWindowParser(ArcStandardParser):

    def __init__(self, vocab_words, vocab_tags, word_dim=50, tag_dim=10, hidden_dim=180):
        # TODO: Add your own code
        super().__init__()

        embedding_specs = [(3, len(vocab_words), word_dim), (3, len(vocab_tags), tag_dim)]

        self.vocab_words = vocab_words
        self.vocab_tags  = vocab_tags
        self.model = FixedWindowModel(embedding_specs, hidden_dim, 3)

    def featurize(self, words, tags, config):
            # TODO: Replace the next line with your own code
        i, stack, heads = config[0], config[1], config[2]
        features = [0, 0, 0, 0, 0, 0]

        if len(words) > i:
              features[0] = words[i]
        if len(stack) > 0:        # Must be 1 
          if len(words) > stack[-1]:
            features[1] = words[stack[-1]]
        if len(stack) > 1:        # Must be 2
          if len(words) > stack[-2]:
           features[2]= words[stack[-2]]
        if len(tags) > i:
          features[3] = tags[i]
        if len(stack) > 0:
          if stack[-1] < len(tags):
            features[4] = tags[stack[-1]]
        if len(stack) > 1:
          if stack[-2] < len(tags):
            features[5] = tags[stack[-2]]
        
        return torch.LongTensor(features)

    def predict(self, words, tags):
        # TODO: Replace the next line with your own code
        config = self.initial_config(len(words))
        word_ids = [self.vocab_words[word] if word in self.vocab_words else 0 for word in words]
        tag_ids = [self.vocab_tags[tag] for tag in tags]
        
        while not self.is_final_config(config):
            valid_moves = self.valid_moves(config)
            
            feature = self.featurize(word_ids, tag_ids, config)
            scores = self.model.forward(feature)
            scores = scores.tolist()

            temp = []
            for move in valid_moves:
              temp.append(scores[move])      

            highest = max(temp)
            config = self.next_config(config, scores.index(highest))
        return config[2]

In [404]:
test = FixedWindowParser(words, tags)
print(example_sentence)
word_inp = [i[0] for i in example_sentence]
tag_inp  = [i[1] for i in example_sentence]
test.predict(word_inp, tag_inp)
#test.predict(example_sentence)

[('<root>', '<root>', 0), ('I', 'PRON', 2), ('like', 'VERB', 0), ('yuor', 'PRON', 4), ('blog', 'NOUN', 2), ('.', 'PUNCT', 2)]


[0, 0, 0, 0, 0, 0]

Complete the skeleton code by implementing the methods of this interface:

**__init__** (*self*, *vocab_words*, *vocab_tags*, *word_dim* = 50, *tag_dim* = 10, *hidden_dim* = 100)

> Creates a new fixed-window model of appropriate dimensions and sets up any other data structures that you consider relevant. The parameters *vocab_words* and *vocab_tags* are the word vocabulary and tag vocabulary. The parameters *word_dim* and *tag_dim* specify the embedding width for the word embeddings and tag embeddings.

**featurize** (*self*, *words*, *tags*, *config*)

> Extracts features from the specified parser state according to the feature model given above. The state is specified in terms of the words in the input sentence (*words*, a list of word ids), their part-of-speech tags (*tags*, a list of tag ids), and the parser configuration proper (*config*, as specified in Problem&nbsp;3).

**predict** (*self*, *words*, *tags*)

> Predicts the list of all heads for the input sentence. This simulates the arc-standard algorithm, calling the move classifier whenever it needs to take a decision. The input sentence is specified in terms of the list of its words (strings) and the list of its tags (strings). Both of these should include the pseudoroot.

#### 💡 Hint on the implementation

In the *predict* function, you must make sure to only execute valid moves. One simple way to do so is to let the fixed-window model predict scores for all moves, and to implement your own, customised argmax operation to find the *valid* move with the highest score.

### Problem 5.3: Generate the training examples

Your next task is to implement a function that generates the training examples for the parser. You will train as usual, using minibatch training.

In [405]:
def training_examples(vocab_words, vocab_tags, gold_data, parser, batch_size=100):
    batch_x, batch_y = [], []
    
    for sentence in gold_data:
        word_ids = []
        tag_ids = []
        gold_heads = []
        #word_ids.append(vocab_words['<pad>'])
        #word_ids.append(vocab_words['<unk>'])
        #tag_ids.append(vocab_tags['<pad>'])
        for word, tag, head in sentence:
            word_ids.append(vocab_words[word])
            tag_ids.append(vocab_tags[tag])
            gold_heads.append(head)

        for config, move in oracle_moves(gold_heads):
            
            feature = parser.featurize(word_ids, tag_ids, config)
            batch_x.append(feature)
            batch_y.append(move)
    
            # If batch contains batch size examples, yield them and empty x and y.
            if len(batch_x) == batch_size:
                yield torch.stack(batch_x, dim=0), torch.LongTensor(batch_y)
                batch_x, batch_y = [], []
    yield torch.stack(batch_x, dim=0), torch.LongTensor(batch_y)

In [406]:
parser = FixedWindowParser(words, tags)
for x, y in training_examples(words, tags, list(train_data), parser):
  print(x.size())
  print(y.size())
  break

torch.Size([100, 6])
torch.Size([100])


Your code should comply with the following specification:

**training_examples** (*vocab_words*, *vocab_tags*, *gold_data*, *tagger*, *batch_size* = 100)

> Iterates through the given *gold_data* (an iterable of parsed sentences), encodes it into word ids and tag ids using the specified vocabularies *vocab_words* and *vocab_tags*, and then yields batches of training examples for gradient-based training. Each batch contains *batch_size* examples, except for the last batch, which may contain fewer examples. Each example in the batch is created by a call to the `featurize` function of the *parser*.

### Problem 5.4: Training loop

The last piece of the puzzle is the training loop. This should be straightforward by now. Complete the skeleton code in the cell below:

In [407]:
import torch.optim as optim
import torch.nn.functional as F

def train_fixed_window(train_data, n_epochs=1, batch_size=100, lr=1e-2):
    vocab_words, vocab_tags = make_vocabs(train_data)

    parser = FixedWindowParser(vocab_words, vocab_tags)
    model = parser.model
  
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
  
    for epoch in range(n_epochs): 
        count = 1
        progress = 0
        train_loss = 0
        avg_loss = 0
        model.train()
        for x, y in training_examples(vocab_words, vocab_tags, train_data, parser, batch_size):
            output = model.forward(x) 
            optimizer.zero_grad()
            
            #Compute the loss
            loss = F.cross_entropy(output,  y) 
            train_loss += loss.item()
            progress += len(y)
            
            #clear_output()
            #print("Loss: ", loss.item())
            print("Running Train loss: " + str(train_loss/count))

            count += 1
            
            #Backward pass
            loss.backward()
            
            #Update the parameter of the model
            optimizer.step()

    return parser

Here is the specification of the training function:

**train_fixed_window** (*train_data*, *n_epochs* = 1, *batch_size* = 100, *lr* = 1e-2)

> Trains a fixed-window parser from a set of training data *train_data* (an iterable over parsed sentences) using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*.

The next code cell trains a tagger and evaluates it on the development data:

In [408]:
parser = train_fixed_window(train_data, n_epochs=1)

Running Train loss: 1.110609531402588
Running Train loss: 1.017438679933548
Running Train loss: 0.9593142867088318
Running Train loss: 0.9078272879123688
Running Train loss: 0.8714091420173645
Running Train loss: 0.8357236087322235
Running Train loss: 0.8209289652960641
Running Train loss: 0.7854009866714478
Running Train loss: 0.7463305625650618
Running Train loss: 0.7350731045007706
Running Train loss: 0.7229951863939111
Running Train loss: 0.7081034208337466
Running Train loss: 0.6856297208712652
Running Train loss: 0.6694294810295105
Running Train loss: 0.64804514447848
Running Train loss: 0.6325576882809401
Running Train loss: 0.6272041850230273
Running Train loss: 0.6143530772791969
Running Train loss: 0.6043312220196975
Running Train loss: 0.6115777537226676
Running Train loss: 0.6059505613077254
Running Train loss: 0.6080171696164391
Running Train loss: 0.6028893887996674
Running Train loss: 0.6079064918061098
Running Train loss: 0.6115321481227874
Running Train loss: 0.6105699

In [409]:
print('{:.4f}'.format(uas(parser, dev_data)))

word_inp = [i[0] for i in example_sentence]
tag_inp  = [i[1] for i in example_sentence]
parser.predict(word_inp, tag_inp)

0.6851


[0, 2, 0, 4, 2, 2]

In [412]:
train_data_re = Dataset('en_ewt-ud-train-projectivized-retagged.conllu')

parser_re = train_fixed_window(train_data_re, n_epochs=1)

Running Train loss: 1.084163784980774
Running Train loss: 0.9863705337047577
Running Train loss: 0.9054020643234253
Running Train loss: 0.8760349750518799
Running Train loss: 0.8404073357582093
Running Train loss: 0.8108064830303192
Running Train loss: 0.8026430095945086
Running Train loss: 0.7612296342849731
Running Train loss: 0.7194682359695435
Running Train loss: 0.7096411764621735
Running Train loss: 0.7104181918230924
Running Train loss: 0.7078515042861303
Running Train loss: 0.6953065899702219
Running Train loss: 0.6797142922878265
Running Train loss: 0.6584234337011973
Running Train loss: 0.6472452152520418
Running Train loss: 0.6407687786747428
Running Train loss: 0.630948018696573
Running Train loss: 0.6263246050006465
Running Train loss: 0.6342336788773537
Running Train loss: 0.6274446362540835
Running Train loss: 0.6262845098972321
Running Train loss: 0.620687916227009
Running Train loss: 0.6286330632865429
Running Train loss: 0.6300397861003876
Running Train loss: 0.628043

In [414]:
dev_data_re = Dataset('en_ewt-ud-dev-retagged.conllu')
print('{:.4f}'.format(uas(parser_re, dev_data_re)))

0.6313


**⚠️ Your submitted notebook must contain output demonstrating at least 68% UAS on the development set.**

## Problem 6: Predicted part-of-speech tags (reflection)

The data that you have used in this lab so far contains gold-standard part-of-speech tags, which makes the evaluation of your parser somewhat misleading: In a practical system (including the baseline for the standard project), one does not have access to gold-standard tags; instead one has to first tag the sentences with an automatic part-of-speech tagger.

The lab directory contains the following alternative versions of the two data for this lab:

* `en_ewt-ud-train-projectivized-retagged.conllu`
* `en_ewt-ud-dev-retagged.conllu`

In each of them, the gold-standard part-of-speech tags have been replaced by part-of-speech tags automatically predicted by the tagger from Lab&nbsp;L4.

Run an experiment to assess the effect that using predicted part-of-speech tags instead of gold-standard tags has on the unlabelled attachment score of your parser. Document your exploration in a short reflection piece (ca. 150&nbsp;words). Respond to the following prompts:

* How did you set up your experiment? What results did you get?

We ran the same experiment with two different models and changed the data to the "-retagged" versions for one of the training sessions. The result was worse for the model trained on the retagged datasets.

* Based on what you know about machine learning, did you expect your results? How do you explain them?

Is is expected that manually entered tags would be more correct in such a task as this one, hence making more sense in terms of the predicted dependencies being worse where that is not the case. Furthermore, the dependancies in the test data, if entered manually, should cohere better to the manually entered tags.

* What did you learn? How, exactly, did you learn it? Why does this learning matter?

We learned what oracles and arc-standard parsing is as well as how to implement, and use these techniques for the task of dependancy parsing. We also learned how combining logical features may improve the results of a trained model.

**🥳 Congratulations on finishing the last lab in this course! 🥳**